# Transfer Learning for Image Regression - EfficientNet

In [ ]:
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pylab as plt
from sklearn.model_selection import train_test_split, StratifiedShuffleSplit
import csv
import os
from fastai.vision.all import *

config = tf.compat.v1.ConfigProto(intra_op_parallelism_threads=2, 
                                  inter_op_parallelism_threads=2, 
                                  allow_soft_placement=True) # TensorFlow config
pd.options.mode.chained_assignment = None # Pandas config

In [ ]:
IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS = 224, 224, 3 # Input dimensions for EfficientNet
BATCH_SIZE = 128

def read_and_decode(filename, reshape_dims):
    # Read an image file to a tensor as a sequence of bytes
    image = tf.io.read_file(filename)
    # Convert the tensor to a 3D uint8 tensor
    image = tf.image.decode_jpeg(image, channels=IMG_CHANNELS)
    # Convert 3D uint8 tensor with values in [0, 1]
    image = tf.image.convert_image_dtype(image, tf.float32)
    # Resize the image to the desired size
    return tf.image.resize(image, reshape_dims)
    
def decode_csv(csv_row):
    record_defaults = ['Id', 'Pawpularity']
    filename, pawpularity = tf.io.decode_csv(csv_row, record_defaults)
    pawpularity = tf.convert_to_tensor(np.float(pawpularity), dtype=tf.float32)
    image = read_and_decode(filename, [IMG_HEIGHT, IMG_WIDTH])
    return image, pawpularity

def show_image(filename):
    image = read_and_decode(filename, [IMG_HEIGHT, IMG_WIDTH])
    plt.imshow(image);
    plt.axis('off');

def training_plot(metrics, history):
    f, ax = plt.subplots(1, len(metrics), figsize=(5*len(metrics), 5))
    for idx, metric in enumerate(metrics):
        ax[idx].plot(history.history[metric], ls='dashed')
        ax[idx].set_xlabel('Epochs')
        ax[idx].set_ylabel(metric)
        ax[idx].plot(history.history['val_'+metric]);
        ax[idx].legend(['train_'+metric, 'val_'+metric])


## Get the data

In [ ]:
path = "../input/petfinder-pawpularity-score/"

data = pd.read_csv(path+"/train.csv") # Dataset for images
data['Id'] = data['Id'].apply(lambda x: path+'train/'+x+'.jpg')
x, y = data.drop(["Id", "Pawpularity"], axis=1), data["Pawpularity"]

### Preliminary pet classification

In [ ]:
# Load a train a ResNet34 model for distinguishing between cats and dogs
# Following work will be carried out for each type of pet

#resnet_path = untar_data(URLs.PETS)/'images'

#def is_cat(x): return x[0].isupper()

#dls = ImageDataLoaders.from_name_func(
#    path, get_image_files(resnet_path), valid_pct=0.2, seed=42, label_func=is_cat, item_tfms=Resize(224))

#learn = cnn_learner(dls, resnet34, metrics=error_rate)
#learn.fine_tune(1)

# Export fine-tuned Resnet 34
#torch.save(learn, '/kaggle/working/resnet34.pkl')

In [ ]:
def is_cat(x): return x[0].isupper()
learner = torch.load('../input/resnet34trainedcatsimages/resnet34.pkl')
data['Is cat']= data['Id'].apply(lambda x: eval(learner.predict(x)[0]))

#cats = data.loc[data['Is cat']==True]
#dogs = data.loc[data['Is cat']==False]

In [ ]:
# Pawpularity distribution by pet type

cats['Pawpularity'].hist(label='Cats', alpha=0.3)
dogs['Pawpularity'].hist(label='Dogs', alpha=0.3)
plt.title('Pawpularity score distribution by pet type')
plt.xlabel('Pawpularity score')
plt.ylabel('Count')
plt.legend(loc='upper right')
plt.grid(False)
plt.show()

In [ ]:
# A trial with only cats

x, y = cats[cats['Pawpularity']<99].drop(["Id", "Pawpularity"], axis=1), cats[cats['Pawpularity']<99]["Pawpularity"]

# Create training, validation and test sets for tabular and image data
# First: test set is created by keeping apart 20% of the dataset
# Second: validation set is created by keeping apart 20% of the remaining dataset
# Third: Training set consists of the remaining samples after test and validation set creation

sssplit = StratifiedShuffleSplit(n_splits=1, test_size=0.2) # Use stratified sampling
for train_index, val_index in sssplit.split(x, y):
    train_img_tmp = cats[cats['Pawpularity']<99].iloc[train_index]
    val_img = cats.iloc[val_index][['Id', 'Pawpularity']]
    
sssplit = StratifiedShuffleSplit(n_splits=1, test_size=0.2)
for train_index, test_index in sssplit.split(train_img_tmp, train_img_tmp['Pawpularity']):
    train_img = train_img_tmp.iloc[train_index][['Id', 'Pawpularity']]
    test_img = train_img_tmp.iloc[test_index][['Id', 'Pawpularity']]
    
    
# Export image sets for futher loading and processing
train_img.to_csv('/kaggle/working/training_img.csv', header=False, index=False)
val_img.to_csv('/kaggle/working/val_img.csv', header=False, index=False)
test_img.to_csv('/kaggle/working/test_img.csv', header=False, index=False)

train_dataset = tf.data.TextLineDataset(
    '/kaggle/working/training_img.csv').map(decode_csv).batch(BATCH_SIZE)

val_dataset = tf.data.TextLineDataset(
    '/kaggle/working/val_img.csv').map(decode_csv).batch(BATCH_SIZE)

test_dataset = tf.data.TextLineDataset(
    '/kaggle/working/test_img.csv').map(decode_csv).batch(BATCH_SIZE)

## Build and train EfficientNet model for transfer learning

In [ ]:
# Keeping EfficientNet architecture and retraining top layers for our purpose only

img_augmentation = tf.keras.models.Sequential([
    tf.keras.layers.RandomRotation(0.2),
    tf.keras.layers.RandomTranslation(height_factor=0.1, width_factor=0.1),
    tf.keras.layers.RandomContrast(0.1),
    tf.keras.layers.RandomZoom(0.2)
])

inputs = tf.keras.layers.Input(shape=(IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS))
x = img_augmentation(inputs)
model = tf.keras.applications.EfficientNetB7(
    include_top=False, input_tensor=inputs,
    weights='../input/efficientnet-pretrain-weights/noisystudent/noisy.student.notop-b7.h5') # Use imagenet pre-trained weights

# freeze pre-trained noisy student weights
model.trainable = False

# rebuild top layer for regression
# 1 unit dense layer with no activation function
x = tf.keras.layers.GlobalAveragePooling2D()(model.output)
x = tf.keras.layers.BatchNormalization()(x)
outputs = tf.keras.layers.Dense(1, activation=None)(x)

In [ ]:
print(model.summary())
tf.keras.utils.plot_model(model, show_shapes=True, show_layer_names=False)

In [ ]:
# Compile model

model = tf.keras.models.Model(inputs, outputs, name='EfficientNet')
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-2),
              loss=tf.keras.losses.MeanSquaredError(),
              metrics=[tf.keras.metrics.RootMeanSquaredError()])

# Train model
history = model.fit(train_dataset, epochs=5, validation_data=val_dataset, batch_size=BATCH_SIZE)

# Plot results
training_plot(['loss', 'root_mean_squared_error'], history)

## Compute predictions and build submission process

In [ ]:
sample_submission = pd.read_csv('../input/petfinder-pawpularity-score/sample_submission.csv')
sample_submission['Id'] = sample_submission['Id'].apply(lambda x: '../input/petfinder-pawpularity-score/test/'+x+'.jpg')
sample_submission.to_csv('/kaggle/working/sample_submission.csv', index=False, header=False)
sample_submission = tf.data.TextLineDataset(
    './sample_submission.csv'
).map(decode_csv).batch(BATCH_SIZE)

# Make predictions with our model
sample_prediction = model.predict(sample_submission)

# Format predictions to output for submission
submission_output = pd.concat(
    [pd.read_csv('../input/petfinder-pawpularity-score/sample_submission.csv').drop('Pawpularity', axis=1),
    pd.DataFrame(sample_prediction)],
    axis=1
)
submission_output.columns = [['Id', 'Pawpularity']]

# Output submission file to csv
submission_output.to_csv('submission.csv', index=False)